In [2]:
import cv2

import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

nPoints = 18

KEYPOINTS = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']

POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16]]



frameDir = "../data/frames/"
frameFiles = ["4nHElVbT3HY_1.json", "4nHElVbT3HY_1_aws.json", "4nHElVbT3HY_2_aws.json", \
         "ITNiqNcl6Mw_1_aws.json", "ITNiqNcl6Mw_2_aws.json", "M_rPhEjym1o_1_aws.json", "M_rPhEjym1o_2_aws.json", \
         "L5mqL7ADEsY_1_aws.json"]

def getPoseLines(file):
    #count = 1
    with open(frameDir + file, "r") as f:
        for line in f:
            yield json.loads(line)
            """
            try:
                yield json.loads(line)
            except:
                print(f"Cant read JSON line {count}, {file}")
            count += 1
            """


def dict_to_intDict(dict_posePoint):
        return {int(k): v for k, v in dict_posePoint.items()}

def getFlattenedPose(poseDict):
    
    flat = []
    
    #--- Check if keys are int. If string, turn the keys to int
    if type(list(poseDict.keys())[0]) == str:
        poseDict = dict_to_intDict(poseDict)
    
    #--- Get coords of key points
    for keypoint in range(0, 18):
        flat.append(poseDict[keypoint])
    
    #--- Convert to numpy array and then flatten the coords list
    flat = np.array(flat).flatten()
    return flat

'''
def getFlattenedPose_str(strPoseDict):
    intPoseDict = dict_to_intDict(strPoseDict)
    flat = []
    
    for keypoint in range(0, 18):
        flat.append(intDict[keypoint])
    
    flat = np.array(flat).flatten()
    return flat
'''

def getValidHeight(poseDict):
    """
    Assumes the pose is upright
    
    valid height = Nose to R-Knee, Nose to L-Knee, Neck to R-Knee, Nect to L-Knee in y coords
    Retrun person valid height
    
    """
    
    if type(list(poseDict.keys())[0]) == str:
        poseDict = dict_to_intDict(poseDict)
    
    nose_y = poseDict[KEYPOINTS.index('Nose')][1]
    neck_y = poseDict[KEYPOINTS.index('Neck')][1]
    rknee_y = poseDict[KEYPOINTS.index('R-Knee')][1]
    lknee_y = poseDict[KEYPOINTS.index('L-Knee')][1]
    
    h1, h2, h3, h4 = (-1, -1, -1, -1)
    
    if nose_y >0:
        if rknee_y >0:
            h1 = abs(nose_y - rknee_y)
            
        if lknee_y >0:
            h2 = abs(nose_y - lknee_y)
            
    if neck_y >0:
        if rknee_y >0:
            h3 = abs(neck_y - rknee_y)
            
        if lknee_y >0:
            h4 = abs(neck_y - lknee_y)
    
    return [h1, h2, h3, h4]

def getOnePose(multi_person):
    """
    multi_person should be a dictionary of person poses
    {"person0": pose0,
     "person1": pose1,
     "person2": pose2
    }
    
    returns one person pose with largest height:
    {"personMax": poseMax}
    """
    hts = []
    persons = []
    for person, pose in multi_person.items():
        persons.append(person)
        hts.append(getValidHeight(pose))
    
    hts = np.array(hts)
    
    maxHtIdx = []
    for i in range(hts.shape[1]):
        maxHtIdx.append(np.argmax(hts[:,i]))
    # get the most frequent index
    freqIdx = (max(map(lambda val: (maxHtIdx.count(val), val), set(maxHtIdx)))[1])
    
    maxHtPerson = persons[freqIdx]
    pose = {maxHtPerson: multi_person[maxHtPerson]}
    
    return pose

#### FOR Testing Code
temp = {"person0": {"0": [222, 112], "1": [228, 136], "2": [205, 136], "3": [199, 169],
                    "4": [206, 198], "5": [247, 137], "6": [253, 175], "7": [245, 184], "8": [214, 200],
                    "9": [207, 254], "10": [213, 301], "11": [238, 206], "12": [236, 254], "13": [214, 292],
                    "14": [221, 105], "15": [229, 105], "16": [214, 107], "17": [237, 106]}}

temp_multi = {"person0": {"0": [379, 136], "1": [378, 152], "2": [361, 152],
                            "3": [347, 176], "4": [347, 207], "5": [394, 152],
                            "6": [400, 176], "7": [394, 183], "8": [362, 198],
                            "9": [353, 237], "10": [369, 263], "11": [385, 199],
                            "12": [378, 239], "13": [362, 284], "14": [378, 129],
                            "15": [386, 129], "16": [370, 129], "17": [387, 129]},
                "person1": {"0": [-1, -1], "1": [626, 152], "2": [612, 152],
                            "3": [611, 168], "4": [620, 175], "5": [637, 151], 
                            "6": [637, 167], "7": [-1, -1], "8": [612, 176], 
                            "9": [621, 190], "10": [612, 216], "11": [629, 176], 
                            "12": [637, 191], "13": [637, 215], "14": [-1, -1], 
                            "15": [-1, -1], "16": [-1, -1], "17": [-1, -1]}
               }

temp2_multi = {"person0": {"0": [377, 129], "1": [370, 152], "2": [354, 152], "3": [339, 182], "4": [362, 198], "5": [385, 152], "6": [386, 183], "7": [378, 168], "8": [354, 199], "9": [354, 238], "10": [331, 277], "11": [370, 199], "12": [385, 232], "13": [354, 255], "14": [370, 128], "15": [378, 128], "16": [362, 130], "17": [-1, -1]}, "person1": {"0": [527, 143], "1": [534, 151], "2": [520, 152], "3": [518, 160], "4": [-1, -1], "5": [542, 151], "6": [550, 160], "7": [550, 175], "8": [526, 168], "9": [526, 176], "10": [526, 192], "11": [541, 168], "12": [-1, -1], "13": [-1, -1], "14": [526, 137], "15": [528, 137], "16": [-1, -1], "17": [534, 137]}, "person3": {"0": [596, 135], "1": [596, 144], "2": [588, 144], "3": [-1, -1], "4": [-1, -1], "5": [604, 144], "6": [604, 152], "7": [597, 153], "8": [589, 160], "9": [567, 176], "10": [573, 192], "11": [597, 160], "12": [598, 169], "13": [-1, -1], "14": [596, 130], "15": [597, 130], "16": [-1, -1], "17": [-1, -1]}}
salsaPoseData = [] #This will become dancers X poses(or frames) X 36
count1 = 0
count2 = 0

fps = 24
fps_analysis = 8
fps_step = 3 #for frame differences/steps

pose_data_by_dancer = []
frame_data_by_dancer = []
vids_by_performace = []
for file in [frameFiles[3]]:
    #lineitems = getLines(file)
    prev_frame = 0
    dancer = []
    frames = []
    video_name = file[:11]
    for line in getPoseLines(file):
        frame_no = line["frame_no"]
        
        #Step1: If time between consective successful frames > 2 secs that is 48
        if prev_frame == 0 or (frame_no - prev_frame)>48:
            # Begin new dancer, but first save previous dancer info if valid
            if len(dancer)>0:
                pose_data_by_dancer.append(dancer)
                frame_data_by_dancer.append(frames)
                vids_by_performace.append(video_name+".mp4")
                
            dancer = []
            frames = []
        
        #Step2: if multi-person frame, get one person
        if line["person_count"] >1:
            dancer_pose = getOnePose(line["pers_coords"])
        else:
            dancer_pose = line["pers_coords"]
        
        #Step3: Flatten the pose
        pose_coords = list(dancer_pose.values())[0]
        
        #Step4: Add pose to dancer
        dancer.append(getFlattenedPose(pose_coords))
        frames.append(frame_no)
        
        #reset
        prev_frame = frame_no
        
print(len(pose_data_by_dancer))
for i in range(len(pose_data_by_dancer)):
    poses = len(pose_data_by_dancer[i])
    #if poses>60
    print(f"Sequence {i+1} has {poses} frames")
"""
#To look at a random sequence of dance
p = np.random.randint(0, len(pose_data_by_dancer)) 
print(pose_data_by_dancer[p])
"""
#print(getFlattenedPose(dict_to_intDict(temp["person0"])))


4
Sequence 1 has 642 frames
Sequence 2 has 664 frames
Sequence 3 has 668 frames
Sequence 4 has 887 frames


'\n#To look at a random sequence of dance\np = np.random.randint(0, len(pose_data_by_dancer)) \nprint(pose_data_by_dancer[p])\n'

In [8]:
def getPoseLines(file):
    #count = 1
    with open(frameDir + file, "r") as f:
        for line in f:
            yield json.loads(line)

count = 0
for line in getPoseLines("ITNiqNcl6Mw_1_aws.json"):
    count += 1
    if count>2838:
        print(list(dancer_pose.values())[0])
print(count)

{'0': [261, 105], '1': [261, 121], '2': [253, 121], '3': [-1, -1], '4': [-1, -1], '5': [275, 121], '6': [276, 144], '7': [277, 160], '8': [253, 160], '9': [245, 184], '10': [253, 215], '11': [268, 160], '12': [261, 191], '13': [277, 216], '14': [-1, -1], '15': [267, 105], '16': [-1, -1], '17': [269, 105]}
{'0': [261, 105], '1': [261, 121], '2': [253, 121], '3': [-1, -1], '4': [-1, -1], '5': [275, 121], '6': [276, 144], '7': [277, 160], '8': [253, 160], '9': [245, 184], '10': [253, 215], '11': [268, 160], '12': [261, 191], '13': [277, 216], '14': [-1, -1], '15': [267, 105], '16': [-1, -1], '17': [269, 105]}
{'0': [261, 105], '1': [261, 121], '2': [253, 121], '3': [-1, -1], '4': [-1, -1], '5': [275, 121], '6': [276, 144], '7': [277, 160], '8': [253, 160], '9': [245, 184], '10': [253, 215], '11': [268, 160], '12': [261, 191], '13': [277, 216], '14': [-1, -1], '15': [267, 105], '16': [-1, -1], '17': [269, 105]}
{'0': [261, 105], '1': [261, 121], '2': [253, 121], '3': [-1, -1], '4': [-1, -1